<a href="https://colab.research.google.com/github/HuyenNguyenHelen/CORD-19-KG/blob/master/QA-models/covid_19_kg_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preliminaries

In [1]:
# If using Google Colab run this cell 

# select tensorflow version for colab 
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf 

print('TensorFlow  version: {}'.format(tf.__version__))

# Get the GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow  version: 1.15.2
Found GPU at: /device:GPU:0


In [3]:
%%capture 
# Install AmpliGraph library
! pip install ampligraph

# Required to visualize embeddings with tensorboard projector, comment out if not required!
! pip install --user tensorboard

# Required to plot text on embedding clusters, comment out if not required!
! pip install --user git+https://github.com/Phlya/adjustText

In [4]:
# All imports used in this tutorial 
%tensorflow_version 1.x
import ampligraph
import numpy as np
import pandas as pd
import tensorflow as tf
from ampligraph.datasets import load_fb15k_237
from ampligraph.evaluation import train_test_split_no_unseen, evaluate_performance, mr_score, mrr_score, hits_at_n_score
from ampligraph.discovery import query_topn, discover_facts, find_clusters
from ampligraph.latent_features import TransE, ComplEx, HolE, DistMult, ConvE, ConvKB
from ampligraph.utils import save_model, restore_model

def display_aggregate_metrics(ranks):
    print('Mean Rank:', mr_score(ranks)) 
    print('Mean Reciprocal Rank:', mrr_score(ranks)) 
    print('Hits@1:', hits_at_n_score(ranks, 1))
    print('Hits@10:', hits_at_n_score(ranks, 10))
    print('Hits@100:', hits_at_n_score(ranks, 100))

print('Ampligraph version: {}'.format(ampligraph.__version__))

Ampligraph version: 1.4.0


In [5]:
from ampligraph.datasets import load_fb15k_237, load_wn18rr, load_yago3_10
import pandas as pd
import requests
import io

## Loading Data 
(a triples dataset)

In [6]:
# Downloading the csv file from your GitHub account
# re_pdf = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/Data/new_triples_with_predefined_relations_pdf_June20_.csv').content
# re_pmc = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/Data/new_triples_with_predefined_relations_pmc_June20_.csv').content
# pdf = pd.read_csv(io.StringIO(re_pdf.decode('utf-8')))
# pmc =  pd.read_csv(io.StringIO(re_pmc.decode('utf-8')))
# dataset = pd.concat([pmc, pdf])

re_dataset = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/Data/all-final-cleaned-triples_3.csv').content
dataset = pd.read_csv(io.StringIO(re_dataset.decode('utf-8')))

print(dataset.columns)
dataset.drop(columns='Unnamed: 0', inplace=True)
# dataset = dataset[['subject','new_relation', 'object' ]]
dataset.head(5)

Index(['Unnamed: 0', 'subject', 'new_relation', 'object'], dtype='object')


,subject,new_relation,object
0,dysfunction,disease_species,child
1,et dysfunction,disease_species,child
2,rsv,disease_disease,virus
3,treatment,treat_procedure_species,child
4,urgency,disease_disease,virus


In [7]:
print('Total triples in the KG:', dataset.shape)

Total triples in the KG: (88815, 3)


In [8]:
print('the number of relations: \n', dataset['new_relation'].value_counts())

the number of relations: 
 disease_disease                    23054
gene_gene                          11007
disease_species                     6409
gene_disease                        6367
disease_gene                        5706
treat_procedure_disease             3794
disease_treat_procedure             3525
gene_treat_procedure                2671
drug_disease                        2594
gene_species                        2351
treat_procedure_gene                2316
treat_procedure_treat_procedure     2156
drug_gene                           2106
disease_symptom                     1707
gene_drug                           1592
drug_drug                           1543
disease_drug                        1515
symptom_disease                     1487
treat_procedure_species             1310
drug_treat_procedure                1167
treat_procedure_drug                1141
drug_species                        1031
symptom_symptom                      688
symptom_species               

## Data exploration

# Data Processing
Partition the triples into 10 portions that will be used for buiding QA systems

In [9]:
# Shuffle the dataset
dataset = dataset.sample(frac=1, axis=1).reset_index(drop=True)
subsets = np.array_split(dataset, 10)  
len(subsets)


10

# Developing QA system

## Developing model

### Create training, validation and test splits

In [10]:
from ampligraph.evaluation import train_test_split_no_unseen
def train_set_split (ds, test_frac = 200, val_frac = 100 ):
  # get the validation set of size 500
  test_train, X_test = train_test_split_no_unseen(ds.values, test_frac, seed=0)

  # get the test set of size 1000 from the remaining triples
  X_train, X_val = train_test_split_no_unseen(test_train, val_frac, seed=0)

  print('Total triples:', dataset.shape)
  print('Size of train:', X_train.shape)
  print('Size of valid:', X_val.shape)
  print('Size of test:', X_test.shape)
  return X_train, X_val, X_test

### Model Training

In [11]:
from ampligraph.latent_features import TransE

def training (train_ds, val_ds, test_ds):
  # Train a KGE model
  model = TransE(k=300, 
                epochs=100, 
                eta=1, 
                loss='multiclass_nll', 
                initializer='xavier', initializer_params={'uniform': False},
                regularizer='LP', regularizer_params= {'lambda': 0.001, 'p': 3},
                optimizer= 'adam', optimizer_params= {'lr': 0.0001}, 
                seed= 0, batches_count= 100, verbose=True)

  model.fit(train_ds)
  # ----------------------
  # Evaluate: 
  # Filtered evaluation with ranking strategy assigning worst rank to break ties
  from ampligraph.utils import save_model, restore_model
  save_model(model, 'TransE.pkl')
  model = restore_model('TransE.pkl')

  # create the filter 
  X_filter = np.concatenate([train_ds, val_ds, test_ds], 0)

  # compute ranks
  ranks = evaluate_performance(test_ds, 
                              model=model, 
                              filter_triples=X_filter)

  # ranks are computed per triple
  print('Test set:', X_test.shape)
  print('Size of ranks:', ranks.shape)

  # Aggregate metrics show the aggregate performance of the model on the test set using a single number
  display_aggregate_metrics(ranks)
  # ----------------------
  
  return display_aggregate_metrics(ranks)

X_train, X_val, X_test = train_set_split(dataset)
scores = training (X_train, X_val, X_test) #dataset.to_numpy()
print('........score: \n {}'.format(scores))

Total triples: (88815, 3)
Size of train: (88515, 3)
Size of valid: (100, 3)
Size of test: (200, 3)


100%|██████████| 200/200 [00:01<00:00, 114.13it/s]


Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 503.3225
Mean Reciprocal Rank: 0.11365353503212151
Hits@1: 0.0675
Hits@10: 0.1975
Hits@100: 0.555
Mean Rank: 503.3225
Mean Reciprocal Rank: 0.11365353503212151
Hits@1: 0.0675
Hits@10: 0.1975
Hits@100: 0.555
........score: 
 None


## Knowledge Discovery

In [12]:
from ampligraph.discovery import query_topn

def QA_retrieve (input, top_n ):
  head, relation1, tail = input[0], input[1], input[2]
  if len(relation1.split('_')) <= 2:
    relation2 = '{}_{}'.format(relation1.split('_')[1], relation1.split('_')[0])
  elif len(relation1.split('_')) == 3:
     relation2 = '{}_{}_{}'.format(relation1.split('_')[1], relation1.split('_')[2], relation1.split('_')[0])
  else:
    raise ValueError('error: relation length issue')
  # restore the previously saved model to save time
  model = restore_model('TransE.pkl')
  if relation1  in dataset['new_relation'].tolist():
    triples1, scores1 = query_topn(model, top_n=top_n, 
                                head=head, 
                                relation=relation1, 
                                tail=tail, 
                                ents_to_consider=None, 
                                rels_to_consider=None)
  else:
    triples1, scores1 = [], []

  if relation2  in dataset['new_relation'].tolist():
    triples2, scores2 = query_topn(model, top_n=top_n, 
                                head=tail, 
                                relation=relation2, 
                                tail=head, 
                                ents_to_consider=None, 
                                rels_to_consider=None)
  else:
    triples2, scores2 = [], []

  if len(triples1) + len(triples2) == top_n:
    if len(triples1) ==0: 
      triples = triples1 + triples2.tolist()
      scores = scores1+ scores2.tolist()
    else:
      triples = triples1.tolist() + triples2
      scores = scores1.tolist() + scores2
  else:
    end_idx = round(top_n/2)
    triples = triples1.tolist()[:end_idx] + triples2.tolist()[:end_idx]
    scores = scores1.tolist()[:end_idx] + scores2.tolist()[:end_idx]
   
  for triple, score in zip(triples, scores):
      print('Score: {} \t {} '.format(score, triple))
  return triples, scores
 


In [13]:


# Query1: What are the genes/proteins related to Covid-19
print ('------------------------------------------------------------')
print('What are the genes/proteins related to Covid-19')
question3 = QA_retrieve (['covid-19', 'disease_gene', None], top_n = 20)

# Query2: What species transmit the covid-19
print ('------------------------------------------------------------')
print('What species transmit the covid-19?')
question4 = QA_retrieve (['covid-19', 'disease_species', None], top_n = 20)

# Query3: what are symptoms of the covid-19
print ('------------------------------------------------------------')
print('what are symptoms of the covid-19?')
question5 = QA_retrieve (['covid-19', 'disease_symptom', None], top_n = 20)

# Query4: what are the possible therapeutic procedure of covid-19??
print ('------------------------------------------------------------')
print('what are the possible therapeutic procedure of covid-19?')
question1 = QA_retrieve (['covid-19', 'disease_treat_procedure', None], top_n = 20)

# Query5: What are potential drugs to treat COVID-19?
print ('------------------------------------------------------------')
print('What are potential drugs to treat COVID-19?')
question6 = QA_retrieve (['covid-19', 'disease_drug', None], top_n = 20)

# Query6: What are related diseases of Covid-19
print ('------------------------------------------------------------')
print('What are related diseases of Covid-19')
question2 = QA_retrieve (['covid-19', 'disease_disease', None], top_n = 20)


------------------------------------------------------------
What are the genes/proteins related to Covid-19
Score: -15.995192527770996 	 ['covid-19', 'disease_gene', 'virus'] 
Score: -16.02608871459961 	 ['covid-19', 'disease_gene', 'rna'] 
Score: -16.388051986694336 	 ['covid-19', 'disease_gene', 'rbd'] 
Score: -16.78095817565918 	 ['covid-19', 'disease_gene', 'rna virus'] 
Score: -16.860557556152344 	 ['covid-19', 'disease_gene', 'vaccine'] 
Score: -16.915414810180664 	 ['covid-19', 'disease_gene', 'viral infection'] 
Score: -17.009151458740234 	 ['covid-19', 'disease_gene', 'dmvs'] 
Score: -17.030353546142578 	 ['covid-19', 'disease_gene', 'infection'] 
Score: -17.087631225585938 	 ['covid-19', 'disease_gene', 'recombinant virus'] 
Score: -17.136219024658203 	 ['covid-19', 'disease_gene', 'recruitment'] 
Score: -16.328567504882812 	 ['critical', 'gene_disease', 'covid-19'] 
Score: -16.50141143798828 	 ['rna', 'gene_disease', 'covid-19'] 
Score: -16.61541748046875 	 ['2009', 'gene_d

# Train model, and Knowledge mining with different % of dataset

In [14]:
ALL = []

for idx in range(len(subsets)):
  outputs = {} 
  outputs['subset_%s' %(idx+1)] = []
  data = subsets[:idx+1]
  data_concat = pd.concat(data)
  print ('i: {} -- len: {}'.format(idx, len(data_concat)))

  # Spliting dataset for training
  X_train, X_val, X_test = train_set_split(data_concat)
  # Training...
  print('Training..........')
  training (X_train, X_val, X_test)

  # Retrieving
  # Query1: What are the genes/proteins related to Covid-19
  print ('------------------------------------------------------------')
  print('What are the genes/proteins related to Covid-19')
  question3 = QA_retrieve (['covid-19', 'disease_gene', None], top_n = 20)
  df3 = pd.DataFrame(zip(question3[0],question3[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df3)

  # Query2: What species transmit the covid-19
  print ('------------------------------------------------------------')
  print('What species transmit the covid-19?')
  question4 = QA_retrieve (['covid-19', 'disease_species', None], top_n = 20)
  df4 = pd.DataFrame(zip(question4[0],question4[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df4)

  # Query3: what are symptoms of the covid-19
  print ('------------------------------------------------------------')
  print('what are symptoms of the covid-19?')
  question5 = QA_retrieve (['covid-19', 'disease_symptom', None], top_n = 20)
  df5 = pd.DataFrame(zip(question5[0],question5[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df5)

  # Query4: what are the possible therapeutic procedure of covid-19??
  print ('------------------------------------------------------------')
  print('what are the possible therapeutic procedure of covid-19?')
  question1 = QA_retrieve (['covid-19', 'disease_treat_procedure', None], top_n = 20)
  df1 = pd.DataFrame(zip(question1[0],question1[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df1)

  #Query5: What are potential drugs to treat COVID-19?
  print ('------------------------------------------------------------')
  print('What are potential drugs to treat COVID-19?')
  question6 = QA_retrieve (['covid-19', 'disease_drug', None], top_n = 20)
  df6 = pd.DataFrame(zip(question6[0],question6[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df6)

  # Query6: What are related diseases of Covid-19
  print ('------------------------------------------------------------')
  print('What are related diseases of Covid-19')
  question2 = QA_retrieve (['covid-19', 'disease_disease', None], top_n = 20)
  df2 = pd.DataFrame(zip(question2[0],question2[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df2)

  ALL.append(outputs)
                     
                     

i: 0 -- len: 8882
Total triples: (88815, 3)
Size of train: (8582, 3)
Size of valid: (100, 3)
Size of test: (200, 3)
Training..........


100%|██████████| 200/200 [00:01<00:00, 177.21it/s]


Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 617.4125
Mean Reciprocal Rank: 0.06999264688508079
Hits@1: 0.035
Hits@10: 0.125
Hits@100: 0.3925
Mean Rank: 617.4125
Mean Reciprocal Rank: 0.06999264688508079
Hits@1: 0.035
Hits@10: 0.125
Hits@100: 0.3925
------------------------------------------------------------
What are the genes/proteins related to Covid-19
Score: -16.436466217041016 	 ['covid-19', 'disease_gene', 'coronaviruses'] 
Score: -16.611312866210938 	 ['covid-19', 'disease_gene', 'virus'] 
Score: -17.211181640625 	 ['covid-19', 'disease_gene', 'pdcov'] 
Score: -17.222482681274414 	 ['covid-19', 'disease_gene', 'reassortant virus'] 
Score: -17.246583938598633 	 ['covid-19', 'disease_gene', 'sars cov'] 
Score: -17.30840301513672 	 ['covid-19', 'disease_gene', 'rna'] 
Score: -17.35699462890625 	 ['covid-19', 'disease_gene', 'rbd'] 
Score: -17.363021850585938 	 ['covid-19', 'disease_gene', 'mhv'] 
Score: -17.433815002441406 	 ['covid-19', 'disease_gene', 'coronavirus'] 
Sc

100%|██████████| 200/200 [00:01<00:00, 153.83it/s]


Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 389.0775
Mean Reciprocal Rank: 0.10692221060370849
Hits@1: 0.0475
Hits@10: 0.2325
Hits@100: 0.5475
Mean Rank: 389.0775
Mean Reciprocal Rank: 0.10692221060370849
Hits@1: 0.0475
Hits@10: 0.2325
Hits@100: 0.5475
------------------------------------------------------------
What are the genes/proteins related to Covid-19
Score: -14.132524490356445 	 ['covid-19', 'disease_gene', 'rna'] 
Score: -14.50717544555664 	 ['covid-19', 'disease_gene', 'virus'] 
Score: -14.62454891204834 	 ['covid-19', 'disease_gene', 'coronaviruses'] 
Score: -14.92310905456543 	 ['covid-19', 'disease_gene', 'rbd'] 
Score: -14.930379867553711 	 ['covid-19', 'disease_gene', 'iav'] 
Score: -14.990982055664062 	 ['covid-19', 'disease_gene', 'sars cov'] 
Score: -15.103971481323242 	 ['covid-19', 'disease_gene', 'ctd'] 
Score: -15.149955749511719 	 ['covid-19', 'disease_gene', 'vaccine'] 
Score: -15.172338485717773 	 ['covid-19', 'disease_gene', 'hcv'] 
Score: -15.24818

...

Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 540.8225
Mean Reciprocal Rank: 0.08925386004450486
Hits@1: 0.0375
Hits@10: 0.19
Hits@100: 0.5275
Mean Rank: 540.8225
Mean Reciprocal Rank: 0.08925386004450486
Hits@1: 0.0375
Hits@10: 0.19
Hits@100: 0.5275
------------------------------------------------------------
What are the genes/proteins related to Covid-19
Score: -15.802433013916016 	 ['covid-19', 'disease_gene', 'rna'] 
Score: -15.826850891113281 	 ['covid-19', 'disease_gene', 'virus'] 
Score: -16.010601043701172 	 ['covid-19', 'disease_gene', 'rbd'] 
Score: -16.53249740600586 	 ['covid-19', 'disease_gene', 'np'] 
Score: -16.58502197265625 	 ['covid-19', 'disease_gene', 'rna virus'] 
Score: -16.644989013671875 	 ['covid-19', 'disease_gene', 'infection'] 
Score: -16.6473331451416 	 ['covid-19', 'disease_gene', 'vaccine'] 
Score: -16.723270416259766 	 ['covid-19', 'disease_gene', 'viral infection'] 
Score: -16.75326156616211 	 ['covid-19', 'disease_gene', 'hrv'] 
Score: -16.839

100%|██████████| 200/200 [00:01<00:00, 115.38it/s]


Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 2963.82
Mean Reciprocal Rank: 0.021348026742152193
Hits@1: 0.0075
Hits@10: 0.0425
Hits@100: 0.1775
Mean Rank: 2963.82
Mean Reciprocal Rank: 0.021348026742152193
Hits@1: 0.0075
Hits@10: 0.0425
Hits@100: 0.1775
------------------------------------------------------------
What are the genes/proteins related to Covid-19
Score: -19.8685359954834 	 ['covid-19', 'disease_gene', 'rna'] 
Score: -19.904823303222656 	 ['covid-19', 'disease_gene', 'scv'] 
Score: -20.085716247558594 	 ['covid-19', 'disease_gene', 'virus'] 
Score: -20.18597412109375 	 ['covid-19', 'disease_gene', 'rbd'] 
Score: -20.357921600341797 	 ['covid-19', 'disease_gene', 'ctd'] 
Score: -20.61846160888672 	 ['covid-19', 'disease_gene', 'tgev infection'] 
Score: -20.623611450195312 	 ['covid-19', 'disease_gene', 'wupyv'] 
Score: -20.665014266967773 	 ['covid-19', 'disease_gene', 'eb'] 
Score: -20.677661895751953 	 ['covid-19', 'disease_gene', 'vaccine'] 
Score: -20.761859893

100%|██████████| 200/200 [00:01<00:00, 111.93it/s]


Test set: (200, 3)
Size of ranks: (200, 2)
Mean Rank: 509.29
Mean Reciprocal Rank: 0.11435124423041795
Hits@1: 0.0675
Hits@10: 0.1975
Hits@100: 0.5575
Mean Rank: 509.29
Mean Reciprocal Rank: 0.11435124423041795
Hits@1: 0.0675
Hits@10: 0.1975
Hits@100: 0.5575
------------------------------------------------------------
What are the genes/proteins related to Covid-19
Score: -15.922067642211914 	 ['covid-19', 'disease_gene', 'virus'] 
Score: -16.046836853027344 	 ['covid-19', 'disease_gene', 'rna'] 
Score: -16.352401733398438 	 ['covid-19', 'disease_gene', 'rbd'] 
Score: -16.70211410522461 	 ['covid-19', 'disease_gene', 'rna virus'] 
Score: -16.796672821044922 	 ['covid-19', 'disease_gene', 'vaccine'] 
Score: -16.840309143066406 	 ['covid-19', 'disease_gene', 'viral infection'] 
Score: -16.968311309814453 	 ['covid-19', 'disease_gene', 'infection'] 
Score: -17.051490783691406 	 ['covid-19', 'disease_gene', 'dmvs'] 
Score: -17.09369468688965 	 ['covid-19', 'disease_gene', 'recombinant viru

In [15]:
ALL

[{'subset_1': [                                         triple      score
   0       [covid-19, disease_gene, coronaviruses] -16.436466
   1               [covid-19, disease_gene, virus] -16.611313
   2               [covid-19, disease_gene, pdcov] -17.211182
   3   [covid-19, disease_gene, reassortant virus] -17.222483
   4            [covid-19, disease_gene, sars cov] -17.246584
   5                 [covid-19, disease_gene, rna] -17.308403
   6                 [covid-19, disease_gene, rbd] -17.356995
   7                 [covid-19, disease_gene, mhv] -17.363022
   8         [covid-19, disease_gene, coronavirus] -17.433815
   9                 [covid-19, disease_gene, hcv] -17.519241
   10                [icu, gene_disease, covid-19] -17.227659
   11               [2009, gene_disease, covid-19] -17.301865
   12                [rna, gene_disease, covid-19] -17.544304
   13           [critical, gene_disease, covid-19] -17.596998
   14                 [er, gene_disease, covid-19] -17.755

In [16]:
from google.colab import drive
drive.mount('/content/drive')
import os
dir_path = '/content/drive/MyDrive/KG-output-3'
isExist = os.path.exists(dir_path)

if not isExist:
  os.makedirs(dir_path)

for i in range(len(ALL)): 
  print('i----------%s'%i)
  dir_s_path = '/content/drive/MyDrive/KG-output-3/subset_%s'%str(i+1)
  isExist_s = os.path.exists(dir_s_path)
  if not isExist_s:
    dir = os.makedirs(dir_s_path)
  for j  in range(len(list(ALL[i].values())[0])):
    print('j----------%s'%j)
    path = '/content/drive/MyDrive/KG-output-3/subset_%s'%str(i+1) +'/query_' + str(j) + '.csv'
    print(path)
    print((list(ALL[i].values()))[0][j])
    with open(path, 'w') as file:
      (list(ALL[i].values()))[0][j].to_csv(file)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
i----------0
j----------0
/content/drive/MyDrive/KG-output-3/subset_1/query_0.csv
                                         triple      score
0       [covid-19, disease_gene, coronaviruses] -16.436466
1               [covid-19, disease_gene, virus] -16.611313
2               [covid-19, disease_gene, pdcov] -17.211182
3   [covid-19, disease_gene, reassortant virus] -17.222483
4            [covid-19, disease_gene, sars cov] -17.246584
5                 [covid-19, disease_gene, rna] -17.308403
6                 [covid-19, disease_gene, rbd] -17.356995
7                 [covid-19, disease_gene, mhv] -17.363022
8         [covid-19, disease_gene, coronavirus] -17.433815
9                 [covid-19, disease_gene, hcv] -17.519241
10                [icu, gene_disease, covid-19] -17.227659
11               [2009, gene_disease, covid-19] -17.301865
12                [rna

# QA EVALUATION

In [17]:
# Load groundtruth file
